## Обучение FastText модели

### Устанавливаем зависимости

In [1]:
# !pip install stop_words
# !pip install pymorphy2
# !pip install compress-fasttext

### Импортируем библиотеки

In [2]:
import os
import sys
import re
import pickle
import random

import numpy as np
import pandas as pd
import sklearn.cluster as cluster
import plotly.express as px
import matplotlib.pyplot as plt
import clusteval
import compress_fasttext

from sklearn.decomposition import PCA
from sklearn.metrics import (
    adjusted_rand_score,
    adjusted_mutual_info_score,
    homogeneity_score,
    completeness_score,
    v_measure_score,
    silhouette_score
)
from scipy.linalg import norm
from scipy.spatial.distance import pdist, squareform
from stop_words import get_stop_words
from gensim.models import FastText

SEED = 2023
VECTOR_SIZE = 16
DATA_PATH = '../data/'
MODEL_PATH = '../nlp_model/'

SCRIPT_DIR = os.path.dirname(os.path.abspath('./jupyter_hb'))
sys.path.append(os.path.dirname(SCRIPT_DIR))

from lib.nlp_utils import Preprocessing
from lib.statictics import calculate_center

### Загрузка данных

In [27]:
# резюме, которые мы соскрапили сами

with open(DATA_PATH + "agro_resumes.pickle", "rb") as f:
    resumes = pickle.load(f)

resumes['text'] = (resumes['name'] + ' ' +
                resumes['gender'] + ' ' +
                resumes['experience'].apply(lambda x: ' '.join(x)) + ' ' +
                resumes['education'].apply(lambda x: ' '.join(x)) + ' ' +
                resumes['name'] + ' ' +
                resumes['tags'].apply(lambda x: ' '.join(x)))

In [3]:
# резюме, предоставленные организатором

with open("../resumes.pickle", "rb") as f:
    new_resumes = pickle.load(f)

new_resumes['text'] = (new_resumes['name'] + ' ' +
                new_resumes['gender'] + ' ' +
                new_resumes['experience'].apply(lambda x: ' '.join(x)) + ' ' +
                new_resumes['education'].apply(lambda x: ' '.join(x)) + ' ' +
                new_resumes['name'] + ' ' +
                new_resumes['tags'].apply(lambda x: ' '.join(x)))

In [4]:
new_resumes.head(2)
print(len(new_resumes))

1002


In [30]:
# а также вакансии, потому что работодатели аккуратнее))

with open(DATA_PATH + "agro_vacances.pickle", "rb") as f:
    vacances = pickle.load(f)

vacances['text'] = (vacances['name'] + ' ' +
                vacances['company'] + ' ' +
                vacances['description'].apply(lambda x: ' '.join(x)) + ' ' +
                vacances['tags'].apply(lambda x: ' '.join(x)))

### Препроцессим тексты

In [31]:
%%time
data = pd.concat(
    [vacances[['text']], resumes[['text']], new_resumes[['text']]]
    ).drop_duplicates()
proc = Preprocessing()
tokens = proc.process_texts(data.sample(n=data.shape[0]), 'text', proc_count=4)

CPU times: user 909 ms, sys: 161 ms, total: 1.07 s
Wall time: 23.5 s


### Формируем корпус для обучения модели

In [32]:
corpus = tokens.tolist()

### Обучаем модель эмбеддингов

In [33]:
%%time
random.seed(SEED)

ft = FastText(
    vector_size=VECTOR_SIZE,
    alpha=0.01,
    window=7,
    sg=1,
    hs=1,
    negative=0
)
ft.build_vocab(corpus)
ft.train(corpus, total_examples=ft.corpus_count, epochs=500)
# Wall time: 1h 39min 35s
# (222640985, 264598000)

CPU times: user 2h 33min, sys: 11.5 s, total: 2h 33min 11s
Wall time: 1h 39min 35s


(222640985, 264598000)

### Сохраняем модель

In [34]:
with open(MODEL_PATH + 'ft_11_18.pikle', 'wb') as f:
    pickle.dump(ft.wv, f)

In [35]:
with open(MODEL_PATH + 'ft_11_18.pikle', 'rb') as f:
    model = pickle.load(f)

### Сжимаем модель

In [37]:
small_model = compress_fasttext.prune_ft_freq(model, pq=False)
small_model.save(MODEL_PATH + 'small_model_11_18')

In [5]:
small_model = compress_fasttext.models.CompressedFastTextKeyedVectors.load(MODEL_PATH + 'small_model_11_18')

### Примеры работы модели

In [6]:
'инженер сервисной службы' in small_model

True

In [7]:
small_model['инженер сервисной службы']

array([ 0.0941208 , -0.18850437,  0.14989014, -0.10419964, -0.05670615,
       -0.07730035,  0.0411265 , -0.31554565, -0.1617296 , -0.0988505 ,
       -0.0078437 , -0.23120524, -0.19533183, -0.13649563,  0.14495279,
       -0.24848294], dtype=float32)

In [8]:
small_model.similarity('инженер сервисной службы', 'технический специалист')

0.7487594

In [9]:
small_model.similarity('главный бухгалтер', 'главбух')

0.78247803

In [10]:
small_model.similarity('главный агроном', 'гл.агроном')

0.93793756

In [11]:
small_model.similarity('агроном', 'агром')

0.6473034

In [12]:
small_model.similarity('главный агроном', 'старший агроном')

0.9420409

In [13]:
small_model.similarity('ветврач', 'ветеринар')

0.901

### Для кластеризации берем первую специальность, если в резюме указали несколько

In [14]:
new_resumes['name'].apply(lambda txt: re.split('[.,/]', txt)[0].strip('./!? '))

0          Специалист сервисно-монтажной службы
1                                инженр механик
2                           Тракторист-машинист
3                             Ветеринарный врач
4        машинист подъемника каротажной станции
                         ...                   
997                                     Агроном
998     Руководитель площадки откорма бройлеров
999                                     Агроном
1000                                 тракторист
1001                                    Агроном
Name: name, Length: 1002, dtype: object

In [15]:
random.seed(SEED)

data = (
    pd.DataFrame
    .from_records(new_resumes)
    .assign(
        one_name=lambda df: df['name'].apply(lambda txt: re.split('[,/.]', txt)[0].strip('./!? '))
    )
    .reset_index()
    .rename(columns={'index': 'id'})
)
proc = Preprocessing()
data['tokens_for_clustering'] = proc.process_texts(data, 'one_name')
clustered_data = (
    data
    .loc[data['tokens_for_clustering'].apply(lambda x: len(x) != 0)]
    .assign(
        ft_vectors=lambda df: df['tokens_for_clustering'].apply(
            lambda txt: np.array([small_model[token] for token in txt]).mean(axis=0)
        )
    )
)[['id', 'one_name', 'ft_vectors']]

ft_vectors = np.concatenate(
    clustered_data['ft_vectors'].values
).reshape(clustered_data.shape[0], -1)

### Кластеризация на 60 кластеров

In [16]:
NUM_CLUSTERS = 60

In [17]:
random.seed(SEED)

clustered_data[f'{NUM_CLUSTERS}_clusters'] = cluster.AgglomerativeClustering(
    n_clusters=NUM_CLUSTERS
).fit_predict(ft_vectors)

stats = calculate_center(clustered_data, f'{NUM_CLUSTERS}_clusters', num=NUM_CLUSTERS)
clustered_data = clustered_data.merge(
    stats,
    how='left',
    on=f'{NUM_CLUSTERS}_clusters'
)

In [18]:
clustered_data.head(2)

,id,one_name,ft_vectors,60_clusters,cluster_60_size,cluster_60_mode,cluster_60_center,cluster_60_center_vector
0,0,Специалист сервисно-монтажной службы,"[0.05563, -0.3306, 0.1272, -0.2637, -0.1674, 0...",41,17,Технический специалист,Специалист сервисно-монтажной службы,"[0.05563, -0.3306, 0.1272, -0.2637, -0.1674, 0..."
1,1,инженр механик,"[0.20347553, -0.2867262, 0.58240426, -0.129347...",7,10,Главный инженер - механик (сельское хозяйство),инженр механик,"[0.20347553, -0.2867262, 0.58240426, -0.129347..."


In [19]:
(
    clustered_data
    .groupby(f'{NUM_CLUSTERS}_clusters', as_index=True)
    .agg(
        cluster_mode=(f'cluster_{NUM_CLUSTERS}_mode', 'min'),
        cluster_center=(f'cluster_{NUM_CLUSTERS}_center', 'min'),
        names_list=('one_name', list),
        cluster_size=(f'cluster_{NUM_CLUSTERS}_size', 'min')
    )
    .sort_values(by=f'cluster_size', ascending=False)
)

,cluster_mode,cluster_center,names_list,cluster_size
60_clusters,,,,
12,Инженер-механик,Инженер,"[Инженер, Инженер, Инженер-механик, инженер, и...",161
2,Агроном,Агроном,"[Агроном, Агроном, Агроном, Агроном, Агроном, ...",119
10,Ветеринарный врач,Ветеринарный врач,"[Ветеринарный врач, Ветеринарный врач, Ветерин...",105
55,Главный агроном,Главный агроном,"[Главный агроном, Главный агроном, главный агр...",44
35,Зоотехник,Зоотехник,"[Зоотехник, Зоотехник, Зоотехник, Зоотехник, З...",42
54,Тракторист-машинист,Тракторист-машинист,"[Тракторист-машинист, Тракторист-машинист, тра...",40
13,Главный ветеринарный врач,Главный ветеринарный врач,"[Главный ветеринарный врач, Старший ветеринарн...",36
8,Машинист экскаватора,Машинист экскаватора,"[Машинист экскаватора, Машинист экскаватора, М...",33
30,Сервисный инженер,Сервисный инженер,"[Сервисный инженер, Сервисный инженер, Сервисн...",25
